In [2]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [37]:
df = pd.read_csv("VizTool_Users.csv")
#print(myData)

This workflow is to visualize NCRMP Viz Tool user data for the United States
1. Filter the data to only include locations within the US

In [38]:
# Get data for only the United States
df = df[df['Country'] == 'United States']
# display the dataframe
print(df)


           Country                Region  Views  Total users  \
0    United States              Maryland    923          263   
1    United States               Florida    663          264   
2    United States            California    382          183   
3    United States                Hawaii    372          159   
4    United States              Virginia    320          264   
5    United States            Washington    258          252   
6    United States        South Carolina    241           74   
7    United States         Massachusetts    164           81   
8    United States  District of Columbia    161           76   
9    United States        North Carolina    150           65   
10   United States              Colorado    127           43   
11   United States                 Texas    121          152   
13   United States              New York    119           74   
14   United States          Pennsylvania    115           24   
15   United States                Oregon

2. Sort values in ascending order by number of users and build bar chart of US states by total number of users

In [39]:
# Sort values in ascending order by number of users
df.sort_values(by='Total users', inplace=True)

#df.sort_values(by= 'Total users', inplace=True)
# Use this for the socio coris data
df = df[df.Region != '(not set)']

# Build bar chart for socio coris data
fig = px.bar(df, x='Region', y='Total users',
             hover_data=['Region','Engagement rate','Engaged sessions','Total users','Views','Average session duration'], color='Engaged sessions', color_continuous_scale=px.colors.sequential.Blues_r, 
             height=400).update_xaxes(categoryorder="total ascending")
#Build bar chart for NCRMP data
#fig = px.bar(df, x='Region', y='Users',
#             hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color='Users',
#             height=400)

#fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)','paper_bgcolor': 'rgba(0,0,0,0)',
#})
#fig.update_layout(font_color="white")
fig.show()

3. Convert long state names to 2 letter abbreviations. Filter out entries with no state or region name.

In [40]:

stateDict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

df = df[df.Region != '(not set)']
print(df)

stateList = []
regionList = df['Region'].tolist()
for region in regionList:
    calculatedState = stateDict[region]
    stateList.append(calculatedState)
#countList = df['Users'].tolist()
countList = df['Total users'].tolist()
#print(countList)

           Country                Region  Views  Total users  \
250  United States               Vermont      1            1   
171  United States          North Dakota      1            1   
153  United States                  Utah      2            2   
147  United States         West Virginia      2            2   
111  United States              Arkansas      4            2   
46   United States                 Idaho     16            3   
77   United States               Montana      8            3   
85   United States          Rhode Island      7            3   
81   United States              Delaware      7            4   
92   United States                Nevada      6            4   
112  United States            New Mexico      4            4   
31   United States              Nebraska     31            5   
98   United States              Oklahoma      5            5   
59   United States                Alaska     13            5   
79   United States         New Hampshire

4. Build a US map of total users per state

In [41]:
#fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color=df['Engagement rate'], scope="usa")

fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Total users'], color=df['Total users'], color_continuous_scale=px.colors.sequential.Blues_r, scope="usa")

fig.show()

5. Build map of total engaged sessions per state

In [28]:
#fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color=df['Engaged sessions'], scope="usa")

fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Total users','Engaged sessions','Average session duration'], color=df['Engagement rate'], color_continuous_scale=px.colors.sequential.ice, scope="usa")

fig.show()

In [46]:
description = df.describe()
print(description)

totalUsers = df['Total users'].sum()
averageEngagementRate = df['Engagement rate'].mean()
#totalUsers = df.Users.sum()
totalSessions = df['Engaged sessions'].sum()
print("TOTAL NUMBER OF USERS: " + str(totalUsers))
print("TOTAL NUMBER OF SESSIONS: " + str(totalSessions))
print("MEAN ENGAGEMENT RATE: " + str(averageEngagementRate))

            Views  Total users  Engaged sessions  Engagement rate  \
count   50.000000    50.000000         50.000000        50.000000   
mean    99.420000    53.440000         28.020000         0.441725   
std    174.043285    79.576727         50.346753         0.238022   
min      1.000000     1.000000          0.000000         0.000000   
25%      9.250000     5.000000          2.250000         0.333333   
50%     26.000000    13.000000          8.000000         0.460956   
75%    118.000000    71.750000         27.000000         0.569048   
max    923.000000   264.000000        283.000000         1.000000   

       Views per user  Sessions per user  Event count per user  Event count  \
count       50.000000          50.000000             50.000000    50.000000   
mean         2.057759           1.418110              6.843881   350.060000   
std          1.162143           0.462489              2.791820   571.275387   
min          0.429688           1.000000              3.000000

This workflow is to visualize the geographic extent of users outside of the United States.

1. Filter out United States data.
2. Create bar chart of Users by country with a region breakdown.

In [49]:
#df = pd.read_csv("VizTool_users.csv")
df = pd.read_csv("VizTool_Users.csv")
#print(myData)

df = df[df.Country != 'United States']
#df = df[df.Region != '(not set)']
#df = df[df.totalUsers >=1]
df = df[df.Country != 'RÃ©union']
df = df[df.Country != 'TÃ¼rkiye']
#df = df[df['Engagement rate']>0.1]
df.sort_values(by='Total users', inplace=True)

#fig = px.bar(df, x='Country', y='Users',
#             hover_data=['Users','Region','Engaged sessions','Engagement rate','Average engagement time'], color='Users',
#             height=400).update_xaxes(categoryorder="total ascending")

fig = px.bar(df, x='Country', y='Total users',
             hover_data=['Total users','Region','Engagement rate'], color='Engagement rate',color_continuous_scale=px.colors.sequential.Blues_r,
             height=400).update_xaxes(categoryorder="total ascending")
fig.show()

3. Use country_converter to update the country names to machine readable country codes.
4. Use the country codes to generate a world map of international users.

In [52]:
import country_converter as coco
cc = coco.CountryConverter()

countryCodeList =[]
#print(df)
#df = df.groupby('Country', as_index=False)['Users'].sum()

sortedData = df.groupby('Country')['Total users'].sum()

#sessionsPerUserDF = df.groupby('Country')['Engaged sessions per user'].mean()

#engagedSessionDF = df.groupby('Country')['Engaged sessions'].sum()
engagedSessionDF = df.groupby('Country')['Engaged sessions'].sum()

engagementRateDF = df.groupby('Country')['Engagement rate'].mean()

#engagementTimeDF = df.groupby('Country')['Average engagement time'].mean()
engagementTimeDF = df.groupby('Country')['Average session duration'].mean()

print(engagementRateDF)
#print(engagedSessionDF)

#sortedDataFrame = pd.DataFrame({'Country':sortedData.index, 'Users':sortedData.values, 'Engaged Sessions':engagedSessionDF.values, 'Engagement Rate':engagementRateDF.values, 'Average session duration':engagementTimeDF.values, 'Engaged Sessions per User':sessionsPerUserDF.values})
sortedDataFrame = pd.DataFrame({'Country':sortedData.index, 'Total users':sortedData.values, 'Engaged sessions':engagedSessionDF.values, 'Engagement rate':engagementRateDF.values, 'Average session duration':engagementTimeDF.values})

sortedDataFrame = sortedDataFrame[sortedDataFrame.Country != 'RÃ©union']
sortedDataFrame = sortedDataFrame[sortedDataFrame.Country != 'TÃ¼rkiye']
#print(sortedDataFrame)

countryList = sortedDataFrame.Country.tolist()
#print(str(countryList))
for country in countryList:
    countryCode = cc.convert(names = country, to = 'ISO3')
    if countryCode not in countryCodeList:
        countryCodeList.append(countryCode)

#numberOfUsersList = sortedDataFrame.Users.tolist()
numberOfUsersList = sortedDataFrame['Total users'].tolist()
i=0
for item in numberOfUsersList:
    print(str(countryList[i])+": " + str(item))
    i+=1

sortedDataFrame = pd.DataFrame({'Country':countryCodeList, 'Total users': numberOfUsersList, 'Engaged sessions':engagedSessionDF.values, 'Engagement rate':engagementRateDF.values, 'Average session duration':engagementTimeDF.values})

fig = px.choropleth(sortedDataFrame, locations=countryCodeList,
                    hover_data=['Total users','Engaged sessions','Average session duration'],
                    color=sortedDataFrame['Engagement rate'],
                    #color=sortedDataFrame['Engaged Sessions'],
                    hover_name=countryList, # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Blues_r,
                    )

fig.update_geos(
    resolution=50,
    #showcoastlines=True, coastlinecolor="RebeccaPurple",
    #showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="LightBlue",
    #showlakes=True, lakecolor="Blue",
    #showrivers=True, rivercolor="Blue"
)

fig.show()


Country
American Samoa          0.650000
Argentina               0.000000
Australia               0.178571
Austria                 0.000000
Barbados                0.666667
                          ...   
United Arab Emirates    0.000000
United Kingdom          0.433333
United States           0.444502
Venezuela               1.000000
Vietnam                 0.555556
Name: Engagement rate, Length: 83, dtype: float64
American Samoa: 10
Argentina: 1
Australia: 49
Austria: 3
Barbados: 2
Belgium: 1
Bermuda: 1
Brazil: 16
Bulgaria: 1
Canada: 18
Caribbean Netherlands: 2
Cayman Islands: 1
Chile: 1
China: 35
Colombia: 13
Costa Rica: 10
Croatia: 1
Curaçao: 1
Cyprus: 2
Czechia: 2
Dominican Republic: 12
Egypt: 4
Ethiopia: 1
Finland: 2
France: 53
French Polynesia: 1
Germany: 22
Greece: 6
Guadeloupe: 1
Guam: 26
Hong Kong: 4
Hungary: 1
India: 14
Indonesia: 24
Ireland: 8
Israel: 3
Italy: 6
Jamaica: 6
Japan: 10
Malaysia: 3
Maldives: 5
Marshall Islands: 1
Martinique: 2
Mexico: 9
Mozambique: 1
Myanmar (

In [41]:
description = sortedDataFrame.describe()
print(description)
totalUsers = sortedDataFrame['Total users'].sum()
totalSessions = sortedDataFrame['Engaged sessions'].sum()
medianEngagementRate = sortedDataFrame['Engagement rate'].median()
meanEngagementRate = sortedDataFrame['Engagement rate'].mean()


print("TOTAL NUMBER OF USERS: " + str(totalUsers))
print("TOTAL NUMBER OF SESSIONS: " + str(totalSessions))
print("Number of countries and territories: " + str(len(countryCodeList)))
print("MEDIAN ENGAGEMENT RATE: " + str(medianEngagementRate))
print("MEAN ENGAGEMENT RATE: " + str(meanEngagementRate))

#print("Mean engagement rate: " + str(meanEngagementRate))

       Total users  Engaged sessions  Engagement rate  \
count    82.000000         82.000000        82.000000   
mean      8.512195          4.243902         0.424347   
std      15.934468          7.259835         0.362593   
min       1.000000          0.000000         0.000000   
25%       1.000000          0.000000         0.000000   
50%       2.500000          1.500000         0.389655   
75%       9.000000          4.000000         0.655000   
max     115.000000         42.000000         1.000000   

       Average session duration  
count                 82.000000  
mean                 220.145296  
std                  309.052005  
min                    2.105587  
25%                    7.530839  
50%                  100.067674  
75%                  330.174973  
max                 1539.417615  
TOTAL NUMBER OF USERS: 698
TOTAL NUMBER OF SESSIONS: 348
Number of countries and territories: 82
MEDIAN ENGAGEMENT RATE: 0.3896551724
MEAN ENGAGEMENT RATE: 0.42434666621333045
